In [11]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from pulp import *
from fractions import Fraction
from itertools import combinations

# Define the hypothesis, conclusion, and conjecture classes
class Hypothesis:
    def __init__(self, statements):
        self.statements = statements

class LinearConclusion:
    def __init__(self, target, inequality, slope, other, intercept):
        self.target = target
        self.inequality = inequality
        self.slope = slope
        self.other = other
        self.intercept = intercept

class LinearConjecture:
    def __init__(self, hypothesis, conclusion, symbol, touch, type="positive integer"):
        self.hypothesis = hypothesis
        self.conclusion = conclusion
        self.symbol = symbol
        self.touch = touch
        self.type = type

    def __repr__(self):
        if self.hypothesis.statements:
            hypothesis_str = " and ".join([f"{self.symbol} is {h}" for h in self.hypothesis.statements])
            return (f"For any {self.type} {self.symbol}, if {hypothesis_str}, then "
                    f"{self.conclusion.target}({self.symbol}) {self.conclusion.inequality} "
                    f"{self.conclusion.slope}*{self.conclusion.other}({self.symbol}) + "
                    f"{self.conclusion.intercept}, with equality on {self.touch} instances.")
        else:
            return (f"For any {self.type} {self.symbol}, "
                    f"{self.conclusion.target}({self.symbol}) {self.conclusion.inequality} "
                    f"{self.conclusion.slope}*{self.conclusion.other}({self.symbol}) + "
                    f"{self.conclusion.intercept}, with equality on {self.touch} instances.")

    def get_sharp_objects(self, df):
        X = df[self.conclusion.other].to_numpy()
        Y = df[self.conclusion.target].to_numpy()
        sharp_indices = df[np.isclose(Y, float(self.conclusion.slope) * X + float(self.conclusion.intercept))].index
        return df.loc[sharp_indices]

    def calculate_distances(self, df):
        X = df[self.conclusion.other].to_numpy()
        Y = df[self.conclusion.target].to_numpy()
        distances = np.abs(Y - (float(self.conclusion.slope) * X + float(self.conclusion.intercept)))
        return distances

def make_upper_linear_conjecture(df, target, other, hypothesis, symbol="n"):
    for hyp in hypothesis:
        df = df[df[hyp] == True]
    X = df[other].to_numpy()
    Y = df[target].to_numpy()

    prob = LpProblem("UpperBoundConjecture", LpMinimize)
    w = LpVariable("w")
    b = LpVariable("b")

    prob += lpSum([w * x + b - y for x, y in zip(X, Y)])

    for x, y in zip(X, Y):
        prob += w * x + b - y >= 0

    prob.solve()

    if w.varValue is None or b.varValue is None:
        return None

    m = Fraction(w.varValue).limit_denominator(10)
    b = Fraction(b.varValue).limit_denominator(10)
    if m == 0:
        return None  # Skip trivial conjectures

    touch = np.sum(np.isclose(Y, float(m) * X + float(b)))

    hypothesis = Hypothesis(hypothesis)
    conclusion = LinearConclusion(target, "<=", m, other, b)

    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_lower_linear_conjecture(df, target, other, hypothesis, symbol="n"):
    for hyp in hypothesis:
        df = df[df[hyp] == True]
    X = df[other].to_numpy()
    Y = df[target].to_numpy()

    prob = LpProblem("LowerBoundConjecture", LpMaximize)
    w = LpVariable("w")
    b = LpVariable("b")

    prob += lpSum([w * x + b - y for x, y in zip(X, Y)])

    for x, y in zip(X, Y):
        prob += w * x + b - y <= 0

    prob.solve()

    if w.varValue is None or b.varValue is None:
        return None

    m = Fraction(w.varValue).limit_denominator(10)
    b = Fraction(b.varValue).limit_denominator(10)
    if m == 0:
        return None  # Skip trivial conjectures

    touch = np.sum(np.isclose(Y, float(m) * X + float(b)))

    hypothesis = Hypothesis(hypothesis)
    conclusion = LinearConclusion(target, ">=", m, other, b)

    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_all_upper_linear_conjectures(df, target, others, properties):
    conjectures = []
    for other in others:
        for k in range(4):  # Considering hypotheses of none, one, two, and three boolean properties
            for prop_comb in combinations(properties, k):
                if other != target:
                    conjecture = make_upper_linear_conjecture(df, target, other, prop_comb)
                    if conjecture:
                        conjectures.append(conjecture)
    return conjectures

def make_all_lower_linear_conjectures(df, target, others, properties):
    conjectures = []
    for other in others:
        for k in range(4):  # Considering hypotheses of none, one, two, and three boolean properties
            for prop_comb in combinations(properties, k):
                if other != target:
                    conjecture = make_lower_linear_conjecture(df, target, other, prop_comb)
                    if conjecture:
                        conjectures.append(conjecture)
    return conjectures

def sort_by_touch_number(conjectures):
    return sorted(conjectures, key=lambda x: x.touch, reverse=True)

def apply_theo_heuristic(conjectures):
    filtered_conjectures = []
    for conj_1 in conjectures:
        is_general = True
        for conj_2 in filtered_conjectures:
            if (conj_1.conclusion.slope == conj_2.conclusion.slope and
                conj_1.conclusion.intercept == conj_2.conclusion.intercept and
                conj_1.conclusion.inequality == conj_2.conclusion.inequality and
                set(conj_1.hypothesis.statements).issubset(set(conj_2.hypothesis.statements))):
                is_general = False
                break
        if is_general:
            filtered_conjectures.append(conj_1)
    return filtered_conjectures

def apply_static_dalmatian_heuristic(df, conjectures):
    filtered_conjectures = []
    for conj in conjectures:
        conj_distances = conj.calculate_distances(df)
        keep_conj = True
        for other_conj in filtered_conjectures:
            other_distances = other_conj.calculate_distances(df)
            if np.all(conj_distances >= other_distances):
                keep_conj = False
                break
        if keep_conj:
            filtered_conjectures.append(conj)
    return filtered_conjectures

def txgraffiti_conjecture_generation(df, targets, invariants, properties):
    conjectures = []
    for target in targets:
        upper_conjectures = make_all_upper_linear_conjectures(df, target, invariants, properties)
        lower_conjectures = make_all_lower_linear_conjectures(df, target, invariants, properties)
        conjectures += upper_conjectures + lower_conjectures

    conjectures = sort_by_touch_number(conjectures)
    conjectures = apply_theo_heuristic(conjectures)
    conjectures = apply_static_dalmatian_heuristic(df, conjectures)

    return conjectures

# Generate synthetic data for integers
n_samples = 1_000
np.random.seed(42)

integers = np.arange(1, n_samples + 1)

# Define properties related to Goldbach's Conjecture, Fermat's Last Theorem, and the Collatz Conjecture
def is_goldbach_composite(n):
    if n <= 2 or n % 2 != 0:
        return False
    for p in range(2, n // 2 + 1):
        if is_prime_number(p) and is_prime_number(n - p):
            return True
    return False

def is_fermat_witness(n, k=3):
    return any([a**k + b**k == n**k for a in range(1, n) for b in range(1, n)])

def collatz_steps(n):
    steps = 0
    while n != 1:
        if n % 2 == 0:
            n = n // 2
        else:
            n = 3 * n + 1
        steps += 1
    return steps

# Define helper functions
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

def is_prime_number(n):
    if n <= 1:
        return False
    for i in range(2, int(np.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

# Generate properties
sum_of_digits = np.array([sum(int(digit) for digit in str(i)) for i in integers])
num_divisors = np.array([len([d for d in range(1, i + 1) if i % d == 0]) for i in integers])
is_prime = np.array([is_prime_number(i) for i in integers])
is_even = integers % 2 == 0
is_goldbach = np.array([is_goldbach_composite(i) for i in integers])
is_fermat = np.array([is_fermat_witness(i) for i in integers])
collatz = np.array([collatz_steps(i) for i in integers])

# Create the DataFrame
data = {
    'value': integers,
    'sum_of_digits': sum_of_digits,
    'num_divisors': num_divisors,
    'prime': is_prime,
    'even': is_even,
    'goldbach': is_goldbach,
    'fermat': is_fermat,
    'collatz_steps': collatz,
    'sqrt(collatz_steps)': np.sqrt(collatz),
    'value^2': integers**2,
    'sum_of_digits^2': sum_of_digits**2,
    'sum_of_digits/num_divisors': sum_of_digits / num_divisors
}
df = pd.DataFrame(data)

# Define the targets, invariants, and properties
targets = ["sum_of_digits", "num_divisors", "collatz_steps", "sqrt(collatz_steps)"]
invariants = ["value", "sum_of_digits", "num_divisors", "value^2", "sqrt(collatz_steps)", "sum_of_digits^2", "sum_of_digits/num_divisors"]
properties = ["prime", "even", "goldbach", "fermat"]

# Generate conjectures using the TxGraffiti algorithm
conjectures = txgraffiti_conjecture_generation(df, targets, invariants, properties)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/AI-discovery-in-mathematics-with-TxGraffiti/env/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/838820e1c2b24cb293e05cdb6e1cc012-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/838820e1c2b24cb293e05cdb6e1cc012-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1005 COLUMNS
At line 3008 RHS
At line 4009 BOUNDS
At line 4012 ENDATA
Problem MODEL has 1000 rows, 2 columns and 2000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1000 (0) rows, 2 (0) columns and 2000 (0) elements
0  Obj 0 Primal inf 704.31033 (1000) Dual inf 946.26298 (2) w.o. free dual inf (0)
4  Obj 22015
Optimal - objective value 22015
Optimal objective 22015 - 4 iterations t

In [12]:
# Print the generated conjectures
for i, conj in enumerate(conjectures[:20]):
    print(f"Conjecture {i+1}. ", conj, "\n")

Conjecture 1.  For any positive integer n, if n is prime, then sum_of_digits(n) <= 2*sum_of_digits/num_divisors(n) + 0, with equality on 168 instances. 

Conjecture 2.  For any positive integer n, collatz_steps(n) >= 29/2*sqrt(collatz_steps)(n) + -105/2, with equality on 15 instances. 

Conjecture 3.  For any positive integer n, num_divisors(n) <= -4/3*sum_of_digits(n) + 48, with equality on 9 instances. 

Conjecture 4.  For any positive integer n, sum_of_digits(n) >= 21/10*sum_of_digits/num_divisors(n) + -11/10, with equality on 7 instances. 

Conjecture 5.  For any positive integer n, num_divisors(n) >= 1/10*sum_of_digits/num_divisors(n) + 9/10, with equality on 7 instances. 

Conjecture 6.  For any positive integer n, sum_of_digits(n) <= 1/4*num_divisors(n) + 25, with equality on 4 instances. 

Conjecture 7.  For any positive integer n, if n is even, then sum_of_digits(n) <= 12/7*sum_of_digits/num_divisors(n) + 150/7, with equality on 3 instances. 

Conjecture 8.  For any positive i